In [1]:
import json
import os
import time
import pickle
import random

from instagramy import InstagramUser
from instagramy import InstagramHashTag
from instagramy import InstagramPost

from nedima.utils import adls_utils
from nedima.utils import groundswell as gs
from nedima.utils import env_setup


In [2]:
secrets = env_setup.load_secrets()
adls_client = adls_utils.initialize_storage_account_ad()
fs_client = adls_client.get_file_system_client(file_system=secrets['azure']['adls_fs_landing'])

In [3]:
#secrets

In [4]:
# 1. "time_posts_lost" : 185
# 2. "n_posts_total" : 58
# 2. "n_posts_trim" : 14
# 3. "n_posts_diff" : 44
# 4. "time_posts_diff" : 335
# Time_last_inspection == (time_posts_diff + time_posts_lost)
# 5. "time_last_inspection" : 450
# 6. "id_ig_session" : "45932114631%3Anri1Psz9XrJWgA%3A9"
# 7. "time_next_inspection" : 360
# 8. "id_filepath" : "2021/02/20/153842_0037.json"
# 9. "id_tag" : "surf"


In [4]:
while(True):
    
    tag_dated, sleep_time = gs.start_inspection_iteration('surf')
    time.sleep(sleep_time)

    tag_latest = gs.inspect_posts()

    dump_dict = gs.structure_inspection_json(tag_latest, tag_dated)
    adls_utils.dump_adls_json(dump_dict, tag_latest, fs_client)

    gs.dump_inspection_snapshot(tag_latest, tag_dated, 'surf')
    print("")

[SNAPSHOT] Next inspection scheduled for 2021-02-21 01:52:30. 0 seconds remaining.


NameError: name 'random' is not defined